In [1]:
!pip install selenium

In [2]:
!pip install pyyaml

In [3]:
import os
import time
import yaml
from datetime import datetime

import pandas as pd
from pymongo import MongoClient


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.options import Options

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [4]:
options = webdriver.FirefoxOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
options.add_argument('start-maximized')
browser = webdriver.Remote(command_executor='http://192.168.65.4:4444', options=options)
print("webdriver ok")

webdriver ok


In [5]:
mc = MongoClient('mongodb://mongo')
db = mc['Kayak']

In [8]:
# fonction qui retourne l'url du site 'kayak.com' selon le pays de départ, d'arrivée, la date de départ (vol simple) et la flexibilité,
# le site classera les options selon l'option 'best'

def get_url_kayak(departure, arrival, departure_date, flexibility_option):
    url = f"https://www.kayak.com/flights/{departure}-{arrival}/{departure_date}-{flexibility_option}?sort=bestflight_a"
    print(f"url for '{arrival}' created successfully!")
    return url

In [9]:
def create_coll(name):
    coll = db.create_collection(name)
    print(f"Collection '{name}' created successfully!")
    return coll

In [10]:
def close_alert():
    # Close cookies pop-up
    accept_cookies_xpath = '//*[@id="m88x"]/div[10]/div/div[1]'

    try:
       browser.find_element(By.XPATH, accept_cookies_xpath).click()
       time.sleep(1)
    except NoSuchElementException:
       pass

In [11]:
def bloc_scraping(inner_id):
    
    close_alert()
    time.sleep(1)
    
    try:
        browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]')
    except NoSuchElementException:
        print(f"I didn't find the element at id={inner_id}, refresh of the page")
        browser.refresh()
    except StaleElementReferenceException:
        print("Stale element id, refresh of the page")
        browser.refresh()
    try:

        price = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="f8F1-price-text"]').text[1:]

        date = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="c9L-i"]/div[1]').text
        day = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="c9L-i"]/div[2]').text

        departure_time = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="VY2U"]/div[1]/span[1]').text
        arrival_time = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="VY2U"]/div[1]/span[3]').text

        duration = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="xdW8"]/div[1]').text

        company = browser.find_element(By.XPATH, f'//div[@data-resultid="{inner_id}"]//div[@class="VY2U"]/div[2]').text
        
    except NoSuchElementException:
        print("I didn't find some element, refresh of the page")
        browser.refresh()
        
    return price, date, day, departure_time, arrival_time, duration, company

In [12]:
def page_scraping(url, coll):
    browser.get(url)
    print(f"We are loading the page at : {url} ")
    time.sleep(5)
#     wait = WebDriverWait(browser, 10)
#     element = wait.until(EC.presence_of_element_located((By.ID, 'nrc6')))

    browser.find_element(By.XPATH, '//div[@class="resultsPaginator"]').click()
    print("Loading more...")
    time.sleep(3)

    close_alert()

    flights = browser.find_elements(By.XPATH, '//div[@class="nrc6"]')  #resultInner
    print(f'We loaded info for {len(flights)} flights')

    for flight in flights:
        try:
            time.sleep(0.5)
            inner_id = flight.get_attribute("data-resultid")  #id
        except:
            print ("I didn't find the result id")
            browser.refresh()
             
        price, date, day, departure_time, arrival_time, duration, company = bloc_scraping(inner_id)
        coll.insert_one({
            'price'     : price,
            'date'      : date,
            'day'       : day,
            'departure' : departure_time,
            'arrival'   : arrival_time,
            'duration'  : duration,
            'company'   : company
        })
    print("We scraped info for the flights and put it in the new collection of 'Kayak' database")
    
    return ("Scraping done")

In [18]:
# TUNIS

tunis = get_url_kayak('PAR', 'TUN', '2023-03-22', 'flexible')
tunis_coll = create_coll('tunis')
page_scraping(tunis, tunis_coll)

url for 'TUN' created successfully!
Collection 'tunis' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-TUN/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 31 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [19]:
print(f"tunis_coll = {tunis_coll.count_documents({})} documents")

tunis_coll = 31 documents


In [22]:
# MONTREAL

montreal = get_url_kayak('PAR', 'YMQ', '2023-03-22', 'flexible')
montreal_coll  = create_coll('montreal')
page_scraping(montreal, montreal_coll)

url for 'YMQ' created successfully!
Collection 'montreal' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-YMQ/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 32 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [23]:
print(f"montreal_coll = {montreal_coll.count_documents({})} documents")

montreal_coll = 32 documents


In [26]:
# BANGKOK

bangkok = get_url_kayak('PAR', 'BKK', '2023-03-22', 'flexible')
bangkok_coll = create_coll('bangkok')
page_scraping(bangkok, bangkok_coll)

url for 'BKK' created successfully!
Collection 'bangkok' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-BKK/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 31 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [27]:
print(f"bangkok_coll = {bangkok_coll.count_documents({})} documents")

bangkok_coll = 31 documents


In [30]:
# NEW YORK

newyork = get_url_kayak('PAR', 'NYC', '2023-03-22', 'flexible')
newyork_coll = create_coll('newyork')
page_scraping(newyork, newyork_coll)

url for 'NYC' created successfully!
Collection 'newyork' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-NYC/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 31 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [31]:
print(f"newyork_coll = {newyork_coll.count_documents({})} documents")

newyork_coll = 31 documents


In [34]:
# MARRAKECH

marrakech = get_url_kayak('PAR', 'RAK', '2023-03-22', 'flexible')
marrakech_coll = create_coll('marrakech')
page_scraping(marrakech, marrakech_coll)

url for 'RAK' created successfully!
Collection 'marrakech' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-RAK/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 30 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [35]:
print(f"marrakech_coll = {marrakech_coll.count_documents({})} documents")

marrakech_coll = 30 documents


In [38]:
# VENICE

venice = get_url_kayak('PAR', 'VCE', '2023-03-22', 'flexible')
venice_coll = create_coll('venice')
page_scraping(venice, venice_coll)

url for 'VCE' created successfully!
Collection 'venice' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-VCE/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 30 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [39]:
print(f"venice_coll = {venice_coll.count_documents({})} documents")

venice_coll = 30 documents


In [42]:
# MADRID

madrid = get_url_kayak('PAR', 'MAD', '2023-03-22', 'flexible')
madrid_coll = create_coll('madrid')
page_scraping(madrid, madrid_coll)

url for 'MAD' created successfully!
Collection 'madrid' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-MAD/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 31 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [43]:
print(f"madrid_coll = {madrid_coll.count_documents({})} documents")

madrid_coll = 31 documents


In [46]:
# LONDON

london = get_url_kayak('PAR', 'LON', '2023-03-22', 'flexible')
london_coll = create_coll('london')
page_scraping(london, london_coll)

url for 'LON' created successfully!
Collection 'london' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-LON/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 30 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [47]:
print(f"london_coll = {london_coll.count_documents({})} documents")

london_coll = 30 documents


In [50]:
# BERLIN

berlin = get_url_kayak('PAR', 'BER', '2023-03-22', 'flexible')
berlin_coll = create_coll('berlin')
page_scraping(berlin, berlin_coll)

url for 'BER' created successfully!
Collection 'berlin' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-BER/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 30 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [51]:
print(f"berlin_coll = {berlin_coll.count_documents({})} documents")

berlin_coll = 30 documents


In [54]:
# OSLO

oslo = get_url_kayak('PAR', 'OSL', '2023-03-22', 'flexible')
oslo_coll = create_coll('oslo')
page_scraping(oslo, oslo_coll)

url for 'OSL' created successfully!
Collection 'oslo' created successfully!
We are loading the page at : https://www.kayak.com/flights/PAR-OSL/2023-03-22-flexible?sort=bestflight_a 
Loading more...
We loaded info for 32 flights
We scraped info for the flights and put it in the new collection of 'Kayak' database


'Scraping done'

In [55]:
print(f"oslo_coll = {oslo_coll.count_documents({})} documents")

oslo_coll = 32 documents


In [56]:
# mc.drop_database('Kayak')
# db.drop_collection('oslo')
print(db.list_collection_names())


['venice', 'london', 'montreal', 'oslo', 'berlin', 'newyork', 'marrakech', 'madrid', 'tunis', 'bangkok']


In [60]:
cursor = marrakech_coll.find()
entries = list(cursor)
entries[:5]

[{'_id': ObjectId('63f94ae653f4a4115483321f'),
  'price': '37',
  'date': '3/21',
  'day': 'Tue',
  'departure': '7:05 pm',
  'arrival': '9:20 pm',
  'duration': '3h 15m',
  'company': 'Ryanair'},
 {'_id': ObjectId('63f94ae853f4a41154833220'),
  'price': '41',
  'date': '3/25',
  'day': 'Sat',
  'departure': '8:45 pm',
  'arrival': '11:00 pm',
  'duration': '3h 15m',
  'company': 'Ryanair'},
 {'_id': ObjectId('63f94aea53f4a41154833221'),
  'price': '42',
  'date': '3/19',
  'day': 'Sun',
  'departure': '8:20 pm',
  'arrival': '10:35 pm',
  'duration': '3h 15m',
  'company': 'Ryanair'},
 {'_id': ObjectId('63f94aec53f4a41154833222'),
  'price': '44',
  'date': '3/24',
  'day': 'Fri',
  'departure': '11:05 am',
  'arrival': '1:20 pm',
  'duration': '3h 15m',
  'company': 'Ryanair'},
 {'_id': ObjectId('63f94aed53f4a41154833223'),
  'price': '47',
  'date': '3/20',
  'day': 'Mon',
  'departure': '10:40 am',
  'arrival': '12:55 pm',
  'duration': '3h 15m',
  'company': 'Ryanair'}]